### Final Script

In [25]:
import pandas as pd
import numpy as np
import glob, os, re, warnings
from pathlib import Path

# ── CONFIG ──────────────────────────────────────────────────────────────────────

# Folder containing all your SDRF Excel files
INPUT_FOLDER = r"D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables"

# Output path for the final CSV
OUTPUT_CSV = r"D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\cleaned_sdrf_with_final_amount.csv"

# District‐column name variants to look for
DISTRICT_COLUMNS = [
    'District','Name of Districts','Name of District',
    'Distt.','Name of Distt.'
]

# The twelve HP districts to extract
HP_DISTRICTS = [
    'Chamba','Hamirpur','Kangra','Kullu','Mandi',
    'Shimla','Solan','Una','Sirmaur','Bilaspur',
    'Lahaul & Spiti','Kinnaur'
]

# Any extra “named” amount columns beyond the 2245-prefix ones
EXTRA_AMOUNT_COLS = [
    "Amount","Total Amount Approved","Amount of 1st Instalment (@30%)",
    "Total Amount Approved (₹)","Already Released Amount (₹)",
    "Amount to be Released (₹)","Total Estimated Amount (In Rs.)",
    "1st Instalment Amount to be Released (₹) @29%",
    "Amount (Rs. in crore)","Total Amount (Rs. in crore)",
    "Total Amount","Amount (Rs. in Crore)","Total (Rs. in Crore)",
    "Total\n(Rs. in crore)","Amount (In Rs crore)",
    "Amount (In Crore)","Total (Rs. in crore)",
    "Funds released (lakh)"
]

# Multipliers for unit keywords in column NAMES (if you ever need them)
UNIT_MULTIPLIERS = {
    'crore': 1e7,
    'lakh':  1e5
}


# ── STEP 1: LOAD & TAG ALL EXCEL SHEETS ─────────────────────────────────────────

def load_and_tag(path: Path) -> list[pd.DataFrame]:
    """
    Reads every sheet in `path`, forcing all cells to strings (so commas stay intact),
    and tags each DataFrame with:
      - file_name
      - sheet_name
      - timeperiod
    """
    xls = pd.ExcelFile(path)
    out = []
    for sheet in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet, dtype=str)
        df['file_name']  = path.name
        df['sheet_name'] = sheet
        df['timeperiod'] = sheet
        out.append(df)
    return out

def load_all_excels_and_report(folder: str):
    all_sheets = []
    district_sheets = []
    total, with_dist = 0, 0

    for fp in sorted(glob.glob(os.path.join(folder, '*.xls*'))):
        for df in load_and_tag(Path(fp)):
            total += 1
            # if any district‐column appears in this sheet, keep it
            if set(df.columns).intersection(DISTRICT_COLUMNS):
                district_sheets.append(df)
                with_dist += 1
            all_sheets.append(df)

    print(f"Found {with_dist}/{total} sheets with district columns")
    if not district_sheets:
        raise RuntimeError("No district‐containing sheets found!")

    district_df = pd.concat(district_sheets, ignore_index=True)
    print("Combined district‐tables shape:", district_df.shape)
    return all_sheets, district_df


# ── STEP 2: CLEAN & UNIFY DISTRICT NAMES ─────────────────────────────────────────

def choose_district(row):
    # pick first non‐null district‐column value, warn if multiple
    vals = [row[c].strip() for c in DISTRICT_COLUMNS
            if c in row and pd.notna(row[c]) and row[c].strip()]
    uniq = []
    for v in vals:
        if v not in uniq:
            uniq.append(v)
    if len(uniq) > 1:
        warnings.warn(f"Row {row.name}: multiple district values {uniq}")
    return uniq[0] if uniq else None

def clean_hp_districts(df: pd.DataFrame, col='district_final') -> pd.DataFrame:
    # extract only the 12 HP districts by regex
    pattern = re.compile(r'\b(' + '|'.join(re.escape(d) for d in HP_DISTRICTS) + r')\b',
                         flags=re.IGNORECASE)
    out = df.copy()
    out['district_final'] = out.apply(choose_district, axis=1)
    out = out.dropna(subset=['district_final'])
    out['district_clean'] = (
        out['district_final']
        .str.extract(pattern, expand=False)
        .str.title()
    )
    return out.dropna(subset=['district_clean'])


# ── STEP 3: AMOUNT PARSING UTILITIES ─────────────────────────────────────────────

def parse_monetary_value(val: str) -> float:
    """
    Keep only digits and dots, so "2,00,00,000" → "20000000" → float(20000000).
    Returns 0.0 on any failure.
    """
    if pd.isna(val):
        return 0.0
    # Remove everything except digits and decimal point
    cleaned = re.sub(r'[^\d\.]', '', val)
    if not cleaned:
        return 0.0
    try:
        return float(cleaned)
    except ValueError:
        return 0.0

def detect_amount_columns(df: pd.DataFrame):
    # any column containing "2245" plus the extras
    detected = [c for c in df.columns if '2245' in c]
    for c in EXTRA_AMOUNT_COLS:
        if c in df.columns and c not in detected:
            detected.append(c)
    if not detected:
        raise RuntimeError("No amount columns detected!")
    return detected

def build_multiplier_map(cols: list[str]) -> dict:
    m = {}
    for col in cols:
        low = col.lower()
        if 'crore' in low:
            m[col] = (UNIT_MULTIPLIERS['crore'], "name contains 'crore'")
        elif 'lakh' in low:
            m[col] = (UNIT_MULTIPLIERS['lakh'],  "name contains 'lakh'")
        else:
            m[col] = (1.0, "no 'crore'/'lakh'; assume rupees")
    return m


# ── MAIN PIPELINE ────────────────────────────────────────────────────────────────

def main():
    # 1) load everything and filter for district sheets
    _, df = load_all_excels_and_report(INPUT_FOLDER)

    # 2) unify district columns
    df['district_final'] = df.apply(choose_district, axis=1)
    df = df[df['district_final'].notna()]
    df = clean_hp_districts(df, col='district_final')

    # 3) detect amount‐cols and build multiplier map
    amt_cols = detect_amount_columns(df)
    mult_map = build_multiplier_map(amt_cols)

    # 4) parse + multiply → per‐column <col>_rupees
    rupee_cols = []
    for col in amt_cols:
        factor, reason = mult_map[col]
        outcol = f"{col}_rupees"
        df[outcol] = df[col].apply(parse_monetary_value) * factor
        rupee_cols.append(outcol)

    # 5) sum into final_amount
    df['final_amount'] = df[rupee_cols].sum(axis=1)

    # 5b) if final_amount is suspiciously small (<100 000), assume it's still in crores
    mask = df['final_amount'] < 100_000
    df.loc[mask, 'final_amount'] *= 10_000_000
    # optional: you can log how many got rescaled
    print(f"Rescaled {mask.sum()} entries of final_amount by ×10 000 000 because they were <100 000")

    # 6) audit trail
    df['multiplier_info'] = ";  ".join(
        f"{col}: ×{mult_map[col][0]} ({mult_map[col][1]})"
        for col in amt_cols
    )

    # 7) save
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"✅ Done! Wrote cleaned data with final_amount to:\n    {OUTPUT_CSV}")


if __name__ == "__main__":
    main()


Found 48/105 sheets with district columns
Combined district‐tables shape: (561, 152)
Rescaled 154 entries of final_amount by ×10 000 000 because they were <100 000
✅ Done! Wrote cleaned data with final_amount to:
    D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\cleaned_sdrf_with_final_amount.csv


C:\Users\saura\AppData\Local\Temp\ipykernel_32916\1064177557.py:167: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['district_final'] = df.apply(choose_district, axis=1)
C:\Users\saura\AppData\Local\Temp\ipykernel_32916\1064177557.py:180: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[outcol] = df[col].apply(parse_monetary_value) * factor
C:\Users\saura\AppData\Local\Temp\ipykernel_32916\1064177557.py:180: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, wh

### Add object-id and clean up columns

In [81]:
import geopandas as gpd

sdrf = pd.read_csv(OUTPUT_CSV)
sdrf['fund_type'] = sdrf['file_name'].str.lower().str[:4]
sdrf_reduced = sdrf[['district_clean', 'final_amount', 'timeperiod','file_name','fund_type']].copy()

hp_tehsil = gpd.read_file(r"D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Himachal-Pradesh\Maps\hp_district_final.geojson")
# Merge the cleaned SDRF data with the tehsil GeoDataFrame
hp_tehsil['District'] = hp_tehsil['District'].str.title()

merged = sdrf_reduced.merge(hp_tehsil[['District','object_id']], left_on='district_clean', right_on='District', how='left')

merged = merged.rename(columns={'final_amount':'relief_and_mitigation_sanction_value', 'timeperiod':'sanction_date'})
merged['timeperiod'] = pd.to_datetime(merged['sanction_date'].str.split('_').str[0], format='%d-%m-%Y').dt.strftime('%Y_%m')
merged.to_csv(r"D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\relief_and_mitigation_funds_final.csv", index=False)

In [83]:
merged

,district_clean,relief_and_mitigation_sanction_value,sanction_date,file_name,fund_type,District,object_id,timeperiod
0,Chamba,50000000.0,14-11-2023,Bank Sanction Order 2023-2024.xlsx,bank,Chamba,02-023,2023_11
1,Hamirpur,40000000.0,14-11-2023,Bank Sanction Order 2023-2024.xlsx,bank,Hamirpur,02-028,2023_11
2,Kangra,30000000.0,14-11-2023,Bank Sanction Order 2023-2024.xlsx,bank,Kangra,02-024,2023_11
3,Kullu,50225400.0,14-11-2023,Bank Sanction Order 2023-2024.xlsx,bank,Kullu,02-026,2023_11
4,Mandi,30000000.0,14-11-2023,Bank Sanction Order 2023-2024.xlsx,bank,Mandi,02-027,2023_11
...,...,...,...,...,...,...,...,...
407,Mandi,2500000.0,01-03-2024,sdrf-sanction-order_2023-24.xlsx,sdrf,Mandi,02-027,2024_03
408,Shimla,7500000.0,01-03-2024,sdrf-sanction-order_2023-24.xlsx,sdrf,Shimla,02-033,2024_03
409,Solan,3500000.0,01-03-2024,sdrf-sanction-order_2023-24.xlsx,sdrf,Solan,02-031,2024_03
410,Una,3991000.0,01-03-2024,sdrf-sanction-order_2023-24.xlsx,sdrf,Una,02-029,2024_03


In [82]:
data_path = r"D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\variables/"
# Total tender variable
variable = 'relief_and_mitigation_sanction_value'
total_expenditure_value_df = merged.groupby(['timeperiod', 'object_id'])[['relief_and_mitigation_sanction_value']].sum().reset_index()

for year_month in merged.timeperiod.unique():
    variable_df_monthly = merged[merged.timeperiod == year_month]
    variable_df_monthly = variable_df_monthly[['object_id', variable]]
    print(data_path)
    if os.path.exists(data_path+'sdrf/'+variable):
        variable_df_monthly.to_csv(data_path+'sdrf/'+variable+'/{}_{}.csv'.format(variable, year_month), index=False)
    else:
        os.mkdir(data_path+'sdrf/'+variable)
        variable_df_monthly.to_csv(data_path+'/sdrf/'+variable+'/{}_{}.csv'.format(variable, year_month), index=False)

D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\variables/
D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\variables/
D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\variables/
D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\variables/
D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\variables/
D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\variables/
D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\variables/
D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\variables/
D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deploymen

### Process

In [16]:
import pandas as pd
import numpy as np
import os
import glob

In [ ]:
# defines district Columns and identifies sheets with those columns and their counts

import glob
import os
from pathlib import Path

district_columns = [
    'District',
    'Name of Districts',
    'Name of District',
    'Name of District/Department',
    'Name of Department / Distt.',
    'Distt.',
    'Name of Distt.'
]

# your existing function (optionally refactored to return counts)
def load_and_tag(path):
    """
    Read every sheet in the given Excel file, tag with its sheet name as timeperiod,
    and return:
      - dfs:   list of DataFrames
      - found: number of sheets containing any district_columns
      - total: total number of sheets
    """
    xls = pd.ExcelFile(path)
    dfs = []
    sheets_with = 0
    total_sheets = len(xls.sheet_names)

    for sheet in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet)
        df['timeperiod'] = sheet
        dfs.append(df)

        if set(df.columns).intersection(district_columns):
            sheets_with += 1

    print(f"{Path(path).name}: {sheets_with}/{total_sheets} sheets have district columns")
    return dfs, sheets_with, total_sheets


def load_all_excels(folder_path):
    """
    Find every .xls/.xlsx in folder_path, run load_and_tag,
    and give you a grand summary at the end.
    """
    excel_paths = glob.glob(os.path.join(folder_path, '*.xls*'))
    all_dfs = []
    grand_with = 0
    grand_total = 0

    for path in sorted(excel_paths):
        dfs, sheets_with, total_sheets = load_and_tag(path)
        all_dfs.extend(dfs)
        grand_with += sheets_with
        grand_total += total_sheets

    print()
    print(f"Overall: {grand_with}/{grand_total} sheets (in {len(excel_paths)} files) contain district columns.")
    return all_dfs


# --- USAGE ---
folder = r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables'
all_dataframes = load_all_excels(folder)


Bank Sanction Order 2023-2024.xlsx: 3/5 sheets have district columns
CB Sanction Order 2023-2024.xlsx: 2/5 sheets have district columns
SDRF-NDRF_2023-24.xlsx: 3/5 sheets have district columns
ndmf-sanction-order_2023-24.xlsx: 3/6 sheets have district columns
ndrf-sanction-order_2022-23.xlsx: 3/6 sheets have district columns
ndrf-sanction-order_2023-24.xlsx: 8/15 sheets have district columns
sdmf-sanction-order_2021-22.xlsx: 0/3 sheets have district columns
sdmf-sanction-order_2022-23.xlsx: 2/3 sheets have district columns
sdrf-sanction-order_2019-2020.xlsx: 4/7 sheets have district columns
sdrf-sanction-order_2020-21.xlsx: 9/16 sheets have district columns
sdrf-sanction-order_2021-22.xlsx: 3/8 sheets have district columns
sdrf-sanction-order_2022-23.xlsx: 5/10 sheets have district columns
sdrf-sanction-order_2023-24.xlsx: 9/16 sheets have district columns

Overall: 54/105 sheets (in 13 files) contain district columns.


In [ ]:
# Defines district columns, tags dataframes with timeperiod, and reports on district columns presence

import pandas as pd
import glob
import os
from pathlib import Path

district_columns = [
    'District',
    'Name of Districts',
    'Name of District',
    'Name of District/Department',
    'Name of Department / Distt.',
    'Distt.',
    'Name of Distt.'
]

def load_and_tag(path):
    """
    Read every sheet in the given Excel file, tag each with its sheet name as 'timeperiod',
    and return the list of DataFrames.
    """
    xls = pd.ExcelFile(path)
    dfs = []
    for sheet in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet)
        df['timeperiod'] = sheet
        dfs.append(df)
    return dfs

def load_all_excels_and_report(folder_path):
    """
    Load every .xls/.xlsx in folder_path, collect all sheets,
    and then print:
      - how many sheets per file have district columns
      - overall percentage of rows containing district info
    """
    excel_paths = glob.glob(os.path.join(folder_path, '*.xls*'))
    all_dfs = []
    grand_with_sheets = 0
    grand_total_sheets = 0

    for path in sorted(excel_paths):
        # load sheets
        dfs = load_and_tag(path)

        # per-file sheet counts
        sheets_with = 0
        for df in dfs:
            if set(df.columns).intersection(district_columns):
                sheets_with += 1
        total_sheets = len(dfs)
        grand_with_sheets += sheets_with
        grand_total_sheets += total_sheets
        print(f"{Path(path).name}: {sheets_with}/{total_sheets} sheets have district columns")

        all_dfs.extend(dfs)

    # concatenate everything
    big_df = pd.concat(all_dfs, ignore_index=True)

    # identify which of your district_columns actually made it in
    found_cols = list(set(big_df.columns).intersection(district_columns))
    if not found_cols:
        print("No district columns found at all!")
        return all_dfs

    # compute row‐level mask
    mask = big_df[found_cols].notna().any(axis=1)
    total_rows = len(big_df)
    rows_with_info = mask.sum()
    pct_with_info = rows_with_info / total_rows * 100

    print()
    print(f"Overall: {grand_with_sheets}/{grand_total_sheets} sheets contain district columns")
    print(f"Across all {total_rows:,} rows, {rows_with_info:,} ({pct_with_info:.2f}%) have district info")

    return all_dfs

# --- USAGE ---
folder = r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables'
all_dataframes = load_all_excels_and_report(folder)




Bank Sanction Order 2023-2024.xlsx: 3/5 sheets have district columns
CB Sanction Order 2023-2024.xlsx: 2/5 sheets have district columns
SDRF-NDRF_2023-24.xlsx: 3/5 sheets have district columns
ndmf-sanction-order_2023-24.xlsx: 3/6 sheets have district columns
ndrf-sanction-order_2022-23.xlsx: 3/6 sheets have district columns
ndrf-sanction-order_2023-24.xlsx: 8/15 sheets have district columns
sdmf-sanction-order_2021-22.xlsx: 0/3 sheets have district columns
sdmf-sanction-order_2022-23.xlsx: 2/3 sheets have district columns
sdrf-sanction-order_2019-2020.xlsx: 4/7 sheets have district columns
sdrf-sanction-order_2020-21.xlsx: 9/16 sheets have district columns
sdrf-sanction-order_2021-22.xlsx: 3/8 sheets have district columns
sdrf-sanction-order_2022-23.xlsx: 5/10 sheets have district columns
sdrf-sanction-order_2023-24.xlsx: 9/16 sheets have district columns

Overall: 54/105 sheets contain district columns
Across all 781 rows, 518 (66.33%) have district info


In [18]:
# identifies district columns, removes potential department columns, tags dataframes with timeperiod, identifies amount columns, 
# and extracts sanction amounts with multipliers for lakhs / crores etc. Also creates a single 'district' column from multiple district columns.
# Exported to a function clean_data(df) that can be applied to the district_df obtained from load_all_excels_and_report.
eda_xls = pd.ExcelFile(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables\ndrf-sanction-order_2022-23.xlsx')
xls = pd.ExcelFile(eda_xls)
dfs = []
for sheet in xls.sheet_names:
        print(sheet)
        df = pd.read_excel(xls, sheet_name=sheet)
        df['timeperiod'] = sheet
        dfs.append(df)

import pandas as pd
import glob
import os
import re
from pathlib import Path

# --- your lookup lists ---
district_columns = [
    'District',
    'Name of Districts',
    'Name of District',
    #'Name of District/Department',
    #'Name of Department / Distt.',
    'Distt.',
    'Name of Distt.'
]
'''
amount_columns = [
    'Total (Rs. in Crore)', 'Amount (Rs. in Crore)',
    'Total\n(Rs. in crore)', 'Amount (In Rs crore)',
    'Amount (In Crore)', 'Amount (Rs. in crore)',
    'Total (Rs. in crore)', 'Funds released (lakh)'
]

multiplier = {
    'crore': 10000000,
    'crores': 10000000,
    'lakhs': 100000,
    'lakh': 100000,
}
'''

def load_and_tag(path: Path) -> list[pd.DataFrame]:
    """
    Read every sheet in the given Excel file, tag each with:
    - 'timeperiod' = sheet name
    - 'file_name' = Excel file name
    - 'sheet_name' = sheet name
    Return the list of DataFrames.
    """
    xls = pd.ExcelFile(path)
    dfs = []
    for sheet in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet, dtype=str)
        df['timeperiod'] = sheet
        df['file_name'] = path.name
        df['sheet_name'] = sheet
        dfs.append(df)
    return dfs

def load_all_excels_and_report(folder_path: str) -> tuple[list[pd.DataFrame], pd.DataFrame]:
    """
    Load every .xls/.xlsx in folder_path, collect all sheets,
    print per-file sheet stats, then:
      - build and return district_df: the concat of only sheets containing district columns
      - also return all_sheets (list of every df) if you need it
    """
    excel_paths = glob.glob(os.path.join(folder_path, '*.xls*'))
    all_sheets: list[pd.DataFrame] = []
    district_sheets: list[pd.DataFrame] = []
    grand_with_sheets = grand_total_sheets = 0

    for path in sorted(excel_paths):
        dfs = load_and_tag(Path(path))

        # count how many sheets in this file have any district column
        sheets_with = 0
        for df in dfs:
            if set(df.columns).intersection(district_columns):
                sheets_with += 1
                district_sheets.append(df)
        total_sheets = len(dfs)
        grand_with_sheets += sheets_with
        grand_total_sheets += total_sheets

        print(f"{Path(path).name}: {sheets_with}/{total_sheets} sheets have district columns")
        all_sheets.extend(dfs)

    # build the final district-only dataframe
    if district_sheets:
        district_df = pd.concat(district_sheets, ignore_index=True)
    else:
        print("No district-containing sheets found!")
        district_df = pd.DataFrame()  # empty

    print(f"\nOverall: {grand_with_sheets}/{grand_total_sheets} sheets contain district columns")
    print(f"Combined district‐tables shape: {district_df.shape}")

    return all_sheets, district_df
'''
def extract_amount(s: pd.Series) -> pd.Series:
    """
    Given a string Series like:
       "Rs. 2.00 Crore", "150 Lakh", "1,234/-", "Rs 2.0 (in Lakhs)", "500"
    returns a float Series with the correct multiplier applied, or NaN.
    """

    # Step 1: pull out the first numeric chunk (with commas or dots)
    num_str = (
        s.str.extract(r'([0-9][0-9\.,]*)', expand=False)    # e.g. "2.00", "150", "1,234"
         .str.replace(',', '', regex=False)                # remove thousands commas
    )

    # convert to float; if it fails, becomes NaN
    num = pd.to_numeric(num_str, errors='coerce')

    # Step 2: pull out a unit keyword, if present
    unit_str = (
        s.str.extract(r'\b(lakh|crore)s?\b', flags=re.IGNORECASE, expand=False)
         .str.lower()
    )

    # map unit to multiplier, default=1
    factor = unit_str.map({'lakh': multiplier['lakh'],
                           'crore': multiplier['crore']})
    factor = factor.fillna(1)

    amt = num * factor

    # Step 3: if num was NaN (no digit sequence found), try a plain cleanup
    mask_missing = amt.isna()
    if mask_missing.any():
        fallback = (
            s[mask_missing]
             .str.replace(r'[^\d\.]', '', regex=True)    # keep only digits+dot
        )
        amt.loc[mask_missing] = pd.to_numeric(fallback, errors='coerce')

    return amt
'''
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # 1) Merge all district_columns into one 'district' column
    dist_cols = [c for c in district_columns if c in df.columns]
    if dist_cols:
        df['district'] = df[dist_cols].bfill(axis=1).iloc[:, 0]
        df.drop(columns=dist_cols, inplace=True)
    '''
    # 2) Build a single sanction_amount by sweeping through every amt_col
    sanction = pd.Series(index=df.index, dtype='float64')
    for col in amount_columns:
        if col in df.columns:
            series = df[col].astype(str).str.strip()
            amt = extract_amount(series)
            # fill only where still missing
            sanction = sanction.fillna(amt)

    df['sanction_amount'] = sanction
    '''
    return df

    # (optional) drop the original amt_cols if you like:
    # df.drop(columns=[c for c in amount_columns if c in df.columns], inplace=True)


# --- USAGE ---
folder = r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables'
all_sheets, district_df = load_all_excels_and_report(folder)

# Now clean only the rows/tables that had district info
cleaned_df = clean_data(district_df)

print("\nCleaned district‐tables shape:", cleaned_df.shape)
cleaned_df.head()


<>:102: SyntaxWarning: invalid escape sequence '\.'
<>:102: SyntaxWarning: invalid escape sequence '\.'
C:\Users\saura\AppData\Local\Temp\ipykernel_32916\4179101711.py:102: SyntaxWarning: invalid escape sequence '\.'
  '''


22-09-2022
22-09-2022_2
22-03-2023
22-03-2023_2
25-03-2023
29-03-2023
Bank Sanction Order 2023-2024.xlsx: 2/5 sheets have district columns
CB Sanction Order 2023-2024.xlsx: 0/5 sheets have district columns
SDRF-NDRF_2023-24.xlsx: 2/5 sheets have district columns
ndmf-sanction-order_2023-24.xlsx: 3/6 sheets have district columns
ndrf-sanction-order_2022-23.xlsx: 3/6 sheets have district columns
ndrf-sanction-order_2023-24.xlsx: 8/15 sheets have district columns
sdmf-sanction-order_2021-22.xlsx: 0/3 sheets have district columns
sdmf-sanction-order_2022-23.xlsx: 2/3 sheets have district columns
sdrf-sanction-order_2019-2020.xlsx: 2/7 sheets have district columns
sdrf-sanction-order_2020-21.xlsx: 9/16 sheets have district columns
sdrf-sanction-order_2021-22.xlsx: 3/8 sheets have district columns
sdrf-sanction-order_2022-23.xlsx: 5/10 sheets have district columns
sdrf-sanction-order_2023-24.xlsx: 9/16 sheets have district columns

Overall: 48/105 sheets contain district columns
Combined dis

,Sr. No.,2245-02-106-01-SOON-NP(OC) (Repair & Restoration of Damaged Road & bridges),2245-02-113-01-SOON-NP(OC) (Repair/Construction of Houses Assistance),2245-02-193-01-SOON-NP(OC) (Assistance to Local Bodies and other Non-Govt Bodies/Inst.),Total,timeperiod,file_name,sheet_name,Account Head,Amount,...,2245-02-101-01-S00N-NP(OC) Cash dole (Immediate relief),2245-02-106-01-S00N-NP(OC) Repairs & Restoration of Damaged roads & Bridges,"2245-02-109-01-S00N-NP(OC) R & R of Damaged Water Supply, Drainage & sewerage works",2245-02-193-01-S00N-NP(OC) Assistance to Local Bodies & other Non-Govt Bodies/Inst.,2245-02-111-01-S00N-NP(OC) Ex-Gratia Payments to Bereaved Families,2245-02-111-02-S00N-NP(OC) Ex-Gratia-II Death Due to State Specific Disaster,2245-02-111-02-S00N-NP(OC) Death Due to State Specific Disaster,2245-02-101-01-S00N-NP(OC) Cash Dole (Immediate Relief),2245-02-106-01-S00N-NP(OC) Repair & Restoration of Damaged Road & Bridges,district
0,1,"5,00,00,000/-",-,-,"5,00,00,000/-",14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chamba
1,2,"2,00,00,000/-","1,00,00,000/-","1,00,00,000/-","4,00,00,000/-",14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hamirpur
2,3,-,"3,00,00,000/-",-,"3,00,00,000/-",14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kangra
3,4,-,"3,00,00,000/-","2,02,25,400/-","5,02,25,400/-",14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kullu
4,5,-,"3,00,00,000/-",-,"3,00,00,000/-",14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mandi


In [19]:
# Cleans up district columns, handles conflicts in district values and extracts sanction amounts

import warnings

def choose_district(row):
    # get values from available district_columns in the row
    vals = [str(row[col]).strip() for col in district_columns if col in row and pd.notnull(row[col]) and str(row[col]).strip()]
    # remove duplicates while preserving order
    unique_vals = []
    for v in vals:
        if v not in unique_vals:
            unique_vals.append(v)
    if len(unique_vals) > 1:
        warnings.warn(f"Conflict in row {row.name}: multiple district values found: {unique_vals}")
    return unique_vals[0] if unique_vals else None

district_df['district_final'] = district_df.apply(choose_district, axis=1)

# Create a single 'district' column from the cleaned district values

district_df_cleaned = district_df.loc[district_df['district_final'].notna(), :].copy()

def clean_hp_districts(df: pd.DataFrame, col: str = 'district_final') -> pd.DataFrame:
    """
    From df[col], extract one of the 12 Himachal Pradesh district names into 'district_clean',
    drop rows without a match, and keep only one row per district.
    """
    # 12 HP districts
    hp_dists = [
        'Chamba','Hamirpur','Kangra','Kullu','Mandi',
        'Shimla','Solan','Una','Sirmaur','Bilaspur',
        'Lahaul & Spiti','Kinnaur'
    ]
    # build a regex to match any district name as a whole word (ignoring case)
    pattern = re.compile(r'\b(' + '|'.join(re.escape(d) for d in hp_dists) + r')\b', flags=re.IGNORECASE)
    
    # Extract the matching district (first group) into a new column
    df = df.copy()
    df['district_clean'] = df[col].str.extract(pattern, expand=False)
    
    # Normalize capitalization (e.g. “kullu” → “Kullu”)
    df['district_clean'] = df['district_clean'].str.title()
    
    # Drop rows where we failed to match any district
    df = df.dropna(subset=['district_clean'])
    
    return df

# suppose your DataFrame is called `df`
sdrf_cleaned = clean_hp_districts(district_df_cleaned, col='district_final')
sdrf_cleaned


C:\Users\saura\AppData\Local\Temp\ipykernel_32916\2473210499.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  district_df['district_final'] = district_df.apply(choose_district, axis=1)


,Sr. No.,Name of Distt.,2245-02-106-01-SOON-NP(OC) (Repair & Restoration of Damaged Road & bridges),2245-02-113-01-SOON-NP(OC) (Repair/Construction of Houses Assistance),2245-02-193-01-SOON-NP(OC) (Assistance to Local Bodies and other Non-Govt Bodies/Inst.),Total,timeperiod,file_name,sheet_name,Name of District,...,2245-02-106-01-S00N-NP(OC) Repairs & Restoration of Damaged roads & Bridges,"2245-02-109-01-S00N-NP(OC) R & R of Damaged Water Supply, Drainage & sewerage works",2245-02-193-01-S00N-NP(OC) Assistance to Local Bodies & other Non-Govt Bodies/Inst.,2245-02-111-01-S00N-NP(OC) Ex-Gratia Payments to Bereaved Families,2245-02-111-02-S00N-NP(OC) Ex-Gratia-II Death Due to State Specific Disaster,2245-02-111-02-S00N-NP(OC) Death Due to State Specific Disaster,2245-02-101-01-S00N-NP(OC) Cash Dole (Immediate Relief),2245-02-106-01-S00N-NP(OC) Repair & Restoration of Damaged Road & Bridges,district_final,district_clean
0,1,Chamba,"5,00,00,000/-",-,-,"5,00,00,000/-",14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chamba,Chamba
1,2,Hamirpur,"2,00,00,000/-","1,00,00,000/-","1,00,00,000/-","4,00,00,000/-",14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hamirpur,Hamirpur
2,3,Kangra,-,"3,00,00,000/-",-,"3,00,00,000/-",14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kangra,Kangra
3,4,Kullu,-,"3,00,00,000/-","2,02,25,400/-","5,02,25,400/-",14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kullu,Kullu
4,5,Mandi,-,"3,00,00,000/-",-,"3,00,00,000/-",14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mandi,Mandi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,8,NaN,NaN,NaN,NaN,"25,00,000/-",01-03-2024,sdrf-sanction-order_2023-24.xlsx,01-03-2024,Mandi,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,–,Mandi,Mandi
554,9,NaN,NaN,NaN,NaN,"75,00,000/-",01-03-2024,sdrf-sanction-order_2023-24.xlsx,01-03-2024,Shimla,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,–,Shimla,Shimla
556,11,NaN,NaN,NaN,NaN,"35,00,000/-",01-03-2024,sdrf-sanction-order_2023-24.xlsx,01-03-2024,Solan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,–,Solan,Solan
557,12,NaN,NaN,NaN,NaN,"39,91,000/-",01-03-2024,sdrf-sanction-order_2023-24.xlsx,01-03-2024,Una,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,–,Una,Una


In [14]:
# identifies district columns, removes potential department columns, tags dataframes with timeperiod, identifies amount columns, 
# and extracts sanction amounts with multipliers for lakhs / crores etc. Also creates a single 'district' column from multiple district columns.
# Exported to a function clean_data(df) that can be applied to the district_df obtained from load_all_excels_and_report.
eda_xls = pd.ExcelFile(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables\ndrf-sanction-order_2022-23.xlsx')
xls = pd.ExcelFile(eda_xls)
dfs = []
for sheet in xls.sheet_names:
        print(sheet)
        df = pd.read_excel(xls, sheet_name=sheet)
        df['timeperiod'] = sheet
        dfs.append(df)

import pandas as pd
import glob
import os
import re
from pathlib import Path

# --- your lookup lists ---
district_columns = [
    'District',
    'Name of Districts',
    'Name of District',
    #'Name of District/Department',
    #'Name of Department / Distt.',
    'Distt.',
    'Name of Distt.'
]

amount_columns = [
    'Total (Rs. in Crore)', 'Amount (Rs. in Crore)',
    'Total\n(Rs. in crore)', 'Amount (In Rs crore)',
    'Amount (In Crore)', 'Amount (Rs. in crore)',
    'Total (Rs. in crore)', 'Funds released (lakh)'
]

multiplier = {
    'crore': 10000000,
    'crores': 10000000,
    'lakhs': 100000,
    'lakh': 100000,
}


def load_and_tag(path: Path) -> list[pd.DataFrame]:
    """
    Read every sheet in the given Excel file, tag each with:
    - 'timeperiod' = sheet name
    - 'file_name' = Excel file name
    - 'sheet_name' = sheet name
    Return the list of DataFrames.
    """
    xls = pd.ExcelFile(path)
    dfs = []
    for sheet in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet, dtype=str)
        df['timeperiod'] = sheet
        df['file_name'] = path.name
        df['sheet_name'] = sheet
        dfs.append(df)
    return dfs

def load_all_excels_and_report(folder_path: str) -> tuple[list[pd.DataFrame], pd.DataFrame]:
    """
    Load every .xls/.xlsx in folder_path, collect all sheets,
    print per-file sheet stats, then:
      - build and return district_df: the concat of only sheets containing district columns
      - also return all_sheets (list of every df) if you need it
    """
    excel_paths = glob.glob(os.path.join(folder_path, '*.xls*'))
    all_sheets: list[pd.DataFrame] = []
    district_sheets: list[pd.DataFrame] = []
    grand_with_sheets = grand_total_sheets = 0

    for path in sorted(excel_paths):
        dfs = load_and_tag(Path(path))

        # count how many sheets in this file have any district column
        sheets_with = 0
        for df in dfs:
            if set(df.columns).intersection(district_columns):
                sheets_with += 1
                district_sheets.append(df)
        total_sheets = len(dfs)
        grand_with_sheets += sheets_with
        grand_total_sheets += total_sheets

        print(f"{Path(path).name}: {sheets_with}/{total_sheets} sheets have district columns")
        all_sheets.extend(dfs)

    # build the final district-only dataframe
    if district_sheets:
        district_df = pd.concat(district_sheets, ignore_index=True)
    else:
        print("No district-containing sheets found!")
        district_df = pd.DataFrame()  # empty

    print(f"\nOverall: {grand_with_sheets}/{grand_total_sheets} sheets contain district columns")
    print(f"Combined district‐tables shape: {district_df.shape}")

    return all_sheets, district_df
'''
def extract_amount(s: pd.Series) -> pd.Series:
    """
    Given a string Series like:
       "Rs. 2.00 Crore", "150 Lakh", "1,234/-", "Rs 2.0 (in Lakhs)", "500"
    returns a float Series with the correct multiplier applied, or NaN.
    """

    # Step 1: pull out the first numeric chunk (with commas or dots)
    num_str = (
        s.str.extract(r'([0-9][0-9\.,]*)', expand=False)    # e.g. "2.00", "150", "1,234"
         .str.replace(',', '', regex=False)                # remove thousands commas
    )

    # convert to float; if it fails, becomes NaN
    num = pd.to_numeric(num_str, errors='coerce')

    # Step 2: pull out a unit keyword, if present
    unit_str = (
        s.str.extract(r'\b(lakh|crore)s?\b', flags=re.IGNORECASE, expand=False)
         .str.lower()
    )

    # map unit to multiplier, default=1
    factor = unit_str.map({'lakh': multiplier['lakh'],
                           'crore': multiplier['crore']})
    factor = factor.fillna(1)

    amt = num * factor

    # Step 3: if num was NaN (no digit sequence found), try a plain cleanup
    mask_missing = amt.isna()
    if mask_missing.any():
        fallback = (
            s[mask_missing]
             .str.replace(r'[^\d\.]', '', regex=True)    # keep only digits+dot
        )
        amt.loc[mask_missing] = pd.to_numeric(fallback, errors='coerce')

    return amt
'''
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # 1) Merge all district_columns into one 'district' column
    dist_cols = [c for c in district_columns if c in df.columns]
    if dist_cols:
        df['district'] = df[dist_cols].bfill(axis=1).iloc[:, 0]
        df.drop(columns=dist_cols, inplace=True)
    '''
    # 2) Build a single sanction_amount by sweeping through every amt_col
    sanction = pd.Series(index=df.index, dtype='float64')
    for col in amount_columns:
        if col in df.columns:
            series = df[col].astype(str).str.strip()
            amt = extract_amount(series)
            # fill only where still missing
            sanction = sanction.fillna(amt)

    df['sanction_amount'] = sanction
    '''
    return df

    # (optional) drop the original amt_cols if you like:
    # df.drop(columns=[c for c in amount_columns if c in df.columns], inplace=True)


# --- USAGE ---
folder = r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables'
all_sheets, district_df = load_all_excels_and_report(folder)

# Now clean only the rows/tables that had district info
cleaned_df = clean_data(district_df)

print("\nCleaned district‐tables shape:", cleaned_df.shape)
cleaned_df.head()


<>:102: SyntaxWarning: invalid escape sequence '\.'
<>:102: SyntaxWarning: invalid escape sequence '\.'
C:\Users\saura\AppData\Local\Temp\ipykernel_32916\4132396634.py:102: SyntaxWarning: invalid escape sequence '\.'
  '''


22-09-2022
22-09-2022_2
22-03-2023
22-03-2023_2
25-03-2023
29-03-2023
Bank Sanction Order 2023-2024.xlsx: 2/5 sheets have district columns
CB Sanction Order 2023-2024.xlsx: 0/5 sheets have district columns
SDRF-NDRF_2023-24.xlsx: 2/5 sheets have district columns
ndmf-sanction-order_2023-24.xlsx: 3/6 sheets have district columns
ndrf-sanction-order_2022-23.xlsx: 3/6 sheets have district columns
ndrf-sanction-order_2023-24.xlsx: 8/15 sheets have district columns
sdmf-sanction-order_2021-22.xlsx: 0/3 sheets have district columns
sdmf-sanction-order_2022-23.xlsx: 2/3 sheets have district columns
sdrf-sanction-order_2019-2020.xlsx: 2/7 sheets have district columns
sdrf-sanction-order_2020-21.xlsx: 9/16 sheets have district columns
sdrf-sanction-order_2021-22.xlsx: 3/8 sheets have district columns
sdrf-sanction-order_2022-23.xlsx: 5/10 sheets have district columns
sdrf-sanction-order_2023-24.xlsx: 9/16 sheets have district columns

Overall: 48/105 sheets contain district columns
Combined dis

,Sr. No.,2245-02-106-01-SOON-NP(OC) (Repair & Restoration of Damaged Road & bridges),2245-02-113-01-SOON-NP(OC) (Repair/Construction of Houses Assistance),2245-02-193-01-SOON-NP(OC) (Assistance to Local Bodies and other Non-Govt Bodies/Inst.),Total,timeperiod,file_name,sheet_name,Account Head,Amount,...,2245-02-101-01-S00N-NP(OC) Cash dole (Immediate relief),2245-02-106-01-S00N-NP(OC) Repairs & Restoration of Damaged roads & Bridges,"2245-02-109-01-S00N-NP(OC) R & R of Damaged Water Supply, Drainage & sewerage works",2245-02-193-01-S00N-NP(OC) Assistance to Local Bodies & other Non-Govt Bodies/Inst.,2245-02-111-01-S00N-NP(OC) Ex-Gratia Payments to Bereaved Families,2245-02-111-02-S00N-NP(OC) Ex-Gratia-II Death Due to State Specific Disaster,2245-02-111-02-S00N-NP(OC) Death Due to State Specific Disaster,2245-02-101-01-S00N-NP(OC) Cash Dole (Immediate Relief),2245-02-106-01-S00N-NP(OC) Repair & Restoration of Damaged Road & Bridges,district
0,1,"5,00,00,000/-",-,-,"5,00,00,000/-",14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chamba
1,2,"2,00,00,000/-","1,00,00,000/-","1,00,00,000/-","4,00,00,000/-",14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hamirpur
2,3,-,"3,00,00,000/-",-,"3,00,00,000/-",14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kangra
3,4,-,"3,00,00,000/-","2,02,25,400/-","5,02,25,400/-",14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kullu
4,5,-,"3,00,00,000/-",-,"3,00,00,000/-",14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mandi


In [22]:
import pandas as pd
import re

# 1) — READ YOUR CSV INTO PANDAS —
#
# Replace this path with wherever you saved your CSV. For example, if you uploaded it
# to /mnt/data/cleaned_districts_sdrf.csv, use that exact path.
df = sdrf_cleaned.copy()  # or pd.read_csv("/path/to/your/file.csv")


# 2) — LIST ALL “MONETARY” COLUMNS YOU WANT TO MERGE INTO final_amount —
# 
# In your file, these were the columns whose names contained 'Amount', 'Total Amount', 'Rs. in crore', etc.
# Feel free to add or remove from this list if you find other SDRF‐related columns later on.
#initial_amount_cols = [col for col in df.columns if re.match(r"^\d", col)]
initial_amount_cols = [col for col in df.columns if "2245" in col]

# 2a) — If you have additional “named” amount columns (that do NOT start with a digit),
#       append them here. For example, if you know “Amount (Rs. in Crore)” is also an SDRF column:


extra_named_amounts = [
    "Amount",
    "Total Amount Approved",
    "Amount of 1st Instalment (@30%)",
    "Total Amount Approved (₹)",
    "Already Released Amount (₹)",
    "Amount to be Released (₹)",
    "Total Estimated Amount (In Rs.)",
    "1st Instalment Amount to be Released (₹) @29%",
    "Amount (Rs. in crore)",
    "Total Amount (Rs. in crore)",
    "Total Amount",
    "Amount (Rs. in Crore)",
    "Total\n(Rs. in crore)",
    "Total",

]

# Only append those extras if they actually exist in your DataFrame:
for col in extra_named_amounts:
    if col in df.columns and col not in initial_amount_cols:
        initial_amount_cols.append(col)

# If you still have a few named columns (like “Amount (Rs. in crore)”) that are NOT digit-headed,
# but should also be treated as “amount” columns, you can append them here. For example:
# extra = ["Amount (Rs. in crore)", "Total Amount Approved"]
# money_cols += extra
#
# But if “every” amount column in your sheet truly starts with a digit, you can leave out any extras.


# 3) — AUTOMATICALLY MAP EACH “money_col” TO A MULTIPLIER BASED ON ITS NAME —
col_to_unit = {}
for col in money_cols:
    lower = col.lower()
    if "crore" in lower:
        # Column name contains “crore” ⇒ multiply parsed value by 1e7
        col_to_unit[col] = {
            "multiplier": 1e7,
            "reason": "column name contains 'crore', so multiply by 1e7"
        }
    elif "lakh" in lower:
        # Column name contains “lakh” ⇒ multiply parsed value by 1e5
        col_to_unit[col] = {
            "multiplier": 1e5,
            "reason": "column name contains 'lakh', so multiply by 1e5"
        }
    else:
        # Otherwise, we assume the raw number (once cleaned) is already in Rupees
        col_to_unit[col] = {
            "multiplier": 1,
            "reason": "no explicit 'crore'/'lakh' in name; assuming value is already in Rupees"
        }


# 4) — HELPER TO STRIP OUT “₹”, COMMAS, “/-”, ETC. AND RETURN A FLOAT —
def parse_monetary_value(val):
    if pd.isna(val):
        return 0.0
    if isinstance(val, str):
        # Remove any character that is not a digit or decimal point
        cleaned = re.sub(r"[^\d\.]", "", val)
        if cleaned == "":
            return 0.0
        try:
            return float(cleaned)
        except ValueError:
            return 0.0
    elif isinstance(val, (int, float)):
        return float(val)
    else:
        return 0.0


# 5) — FOR EACH “money_col”, PARSE + MULTIPLY → STORE IN A NEW “<col>_rupees” COLUMN —
for col, info in col_to_unit.items():
    # 5a) parse each cell into a float
    parsed = df[col].apply(parse_monetary_value)
    # 5b) apply that column’s multiplier
    df[f"{col}_rupees"] = parsed * info["multiplier"]


# 6) — SUM ALL “<col>_rupees” COLUMNS ROW-WISE INTO “final_amount” —
rupee_cols = [f"{col}_rupees" for col in money_cols]
df["final_amount"] = df[rupee_cols].sum(axis=1)


# 7) — BUILD A “multiplier_info” STRING COLUMN FOR AUDIT PURPOSES —
mult_info_list = []
for col, info in col_to_unit.items():
    m    = info["multiplier"]
    why  = info["reason"]
    mult_info_list.append(f"{col}: ×{m} ({why})")

df["multiplier_info"] = ";  ".join(mult_info_list)


# 8) — OPTIONAL: SAVE TO A NEW CSV —
#
#    If you want to keep your original untouched, write out a brand‐new file with final_amount + multiplier_info.
#    (You can always drop the intermediate “_rupees” columns afterward or keep them for debugging.)
#
df

NameError: name 'money_cols' is not defined

In [ ]:
sdrf_cleaned

In [ ]:
import pandas as pd
import re

# 1) — READ YOUR DATAFRAME —
# If you already have your cleaned SDRF DataFrame in a variable called `sdrf_cleaned`, use:
df = sdrf_cleaned.copy()
# Otherwise, uncomment and update the following line to read from CSV:
# df = pd.read_csv("/mnt/data/cleaned_districts_sdrf.csv")


# 2) — DETECT ALL “INITIAL AMOUNT” COLUMNS —
#    (a) Any column whose name starts with a digit
initial_amount_cols = [col for col in df.columns if "2245" in col]

#    (b) Plus any extra named columns you know are monetary fields
extra_named_amounts = [
    "Amount",
    "Total Amount Approved",
    "Amount of 1st Instalment (@30%)",
    "Total Amount Approved (₹)",
    "Already Released Amount (₹)",
    "Amount to be Released (₹)",
    "Total Estimated Amount (In Rs.)",
    "1st Instalment Amount to be Released (₹) @29%",
    "Amount (Rs. in crore)",
    "Total Amount (Rs. in crore)",
    "Total Amount",
    "Amount (Rs. in Crore)",
    "Total (Rs. in Crore)",
    "Total\n(Rs. in crore)",
    "Amount (In Rs crore)",
    "Amount (In Crore)",
    "Total (Rs. in crore)",
    "Funds released (lakh)"
]


for col in extra_named_amounts:
    if col in df.columns and col not in initial_amount_cols:
        initial_amount_cols.append(col)
print(initial_amount_cols)

# 3) — MAP EACH AMOUNT COLUMN TO A MULTIPLIER BASED ON ITS NAME —
col_to_unit = {}
for col in initial_amount_cols:
    low = col.lower()
    if "crore" in low:
        col_to_unit[col] = {
            "multiplier": 1e7,
            "reason": "column name contains 'crore', so multiply by 1e7"
        }
    elif "lakh" in low:
        col_to_unit[col] = {
            "multiplier": 1e5,
            "reason": "column name contains 'lakh', so multiply by 1e5"
        }
    else:
        col_to_unit[col] = {
            "multiplier": 1,
            "reason": "no 'crore'/'lakh' in name; assume already in Rupees"
        }


# 4) — HELPER: CLEAN A RAW CELL INTO A FLOAT (STRIPPING “₹”, COMMAS, ETC.) —
def parse_monetary_value(val):
    if pd.isna(val):
        return 0.0
    if isinstance(val, str):
        numbers = re.findall(r'\d+\.?\d*', val)
        if numbers:
            return float(numbers[0])
        else:
            return 0.0
    return float(val)


# 5) — FOR EACH INITIAL AMOUNT COLUMN → PARSE + MULTIPLY → NEW “<col>_rupees” COLUMN —
new_rupee_cols = []
for col in initial_amount_cols:
    multiplier = col_to_unit[col]["multiplier"]
    parsed = df[col].apply(parse_monetary_value)
    rupee_col = f"{col}_rupees"
    df[rupee_col] = parsed * multiplier
    new_rupee_cols.append(rupee_col)


# 6) — SUM ALL “<col>_rupees” COLUMNS ROW‐WISE INTO “final_amount” —
df["final_amount"] = df[new_rupee_cols].sum(axis=1)


# 7) — BUILD A “multiplier_info” COLUMN FOR AUDIT PURPOSES —
mult_info = [
    f"{col}: ×{info['multiplier']} ({info['reason']})"
    for col, info in col_to_unit.items()
]
df["multiplier_info"] = ";  ".join(mult_info)


# 8) — (OPTIONAL) SAVE TO A NEW CSV —
# df.to_csv("/mnt/data/cleaned_districts_sdrf_with_final_amount.csv", index=False)


['2245-02-106-01-SOON-NP(OC) (Repair & Restoration of Damaged Road & bridges)', '2245-02-113-01-SOON-NP(OC) (Repair/Construction of Houses Assistance)', '2245-02-193-01-SOON-NP(OC) (Assistance to Local Bodies and other Non-Govt Bodies/Inst.)', '2245-02-113-01-SOON-NP(OC) Repair/Construction of Houses Assistance', '2245-02-193-01-SOON-NP(OC) Assistance to Local Bodies and other Non-Govt Bodies/Inst.', '05-2245-02-101-01-C00N-NP-(OC) Cash Dole (Immediate relief)', '05-2245-02-106-01-C00N-NP-(OC) Repair & Restoration of Damaged roads and Bridges', '05-2245-02-109-01-C00N-NP(OC) Repair & Restoration of Damaged Water Supply, Drainage and sewarage works', '05-2245-02-111-02-C00N-NP(OC) Ex-Gratia-II (Death Due to State Specific Disaster)', '05-2245-02-113-01-C00N-NP(OC) Repair/Construction of House Assistance', '05-2245-02-193-01-C00N-NP(OC) Assistance to Local Bodies and other Non-Govt Bodies/Inst.', '05-2245-02-106-01-C00N-NP-(OC) Repair & Restoration of Damaged Roads and Bridges', '05-2245

C:\Users\saura\AppData\Local\Temp\ipykernel_32916\1820258361.py:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[rupee_col] = parsed * multiplier
C:\Users\saura\AppData\Local\Temp\ipykernel_32916\1820258361.py:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[rupee_col] = parsed * multiplier
C:\Users\saura\AppData\Local\Temp\ipykernel_32916\1820258361.py:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining al

In [22]:
df_incorrect = df.loc[df['final_amount']<=30,:].copy()
df_incorrect = df_incorrect.dropna(axis=1, how='all')
df_incorrect.columns.to_list()#correct.shape
df_incorrect_reduced = df_incorrect[['district_clean', 'timeperiod', 'file_name', 'sheet_name', 'final_amount', 'multiplier_info']]
df_incorrect_reduced

,district_clean,timeperiod,file_name,sheet_name,final_amount,multiplier_info
0,Chamba,14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,5.0,2245-02-106-01-SOON-NP(OC) (Repair & Restorati...
1,Hamirpur,14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,4.0,2245-02-106-01-SOON-NP(OC) (Repair & Restorati...
2,Kangra,14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,3.0,2245-02-106-01-SOON-NP(OC) (Repair & Restorati...
3,Kullu,14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,5.0,2245-02-106-01-SOON-NP(OC) (Repair & Restorati...
4,Mandi,14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,3.0,2245-02-106-01-SOON-NP(OC) (Repair & Restorati...
...,...,...,...,...,...,...
543,Kangra,23-01-2024_2,sdrf-sanction-order_2023-24.xlsx,23-01-2024_2,2.0,2245-02-106-01-SOON-NP(OC) (Repair & Restorati...
548,Hamirpur,01-03-2024,sdrf-sanction-order_2023-24.xlsx,01-03-2024,1.0,2245-02-106-01-SOON-NP(OC) (Repair & Restorati...
550,Kinnaur,01-03-2024,sdrf-sanction-order_2023-24.xlsx,01-03-2024,25.0,2245-02-106-01-SOON-NP(OC) (Repair & Restorati...
552,Lahaul & Spiti,01-03-2024,sdrf-sanction-order_2023-24.xlsx,01-03-2024,25.0,2245-02-106-01-SOON-NP(OC) (Repair & Restorati...


In [ ]:
df_incorrect_reduced

In [6]:
df_correct = df.loc[df['final_amount']>=30,:].copy()
df_correct

,Sr. No.,Name of Distt.,2245-02-106-01-SOON-NP(OC) (Repair & Restoration of Damaged Road & bridges),2245-02-113-01-SOON-NP(OC) (Repair/Construction of Houses Assistance),2245-02-193-01-SOON-NP(OC) (Assistance to Local Bodies and other Non-Govt Bodies/Inst.),Total,timeperiod,file_name,sheet_name,Name of District,...,Already Released Amount (₹)_rupees,Amount to be Released (₹)_rupees,Total Estimated Amount (In Rs.)_rupees,1st Instalment Amount to be Released (₹) @29%_rupees,Amount (Rs. in crore)_rupees,Total Amount (Rs. in crore)_rupees,Total Amount_rupees,Amount (Rs. in Crore)_rupees,final_amount,multiplier_info
0,1,Chamba,"5,00,00,000/-",-,-,"5,00,00,000/-",14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50000000.0,2245-02-106-01-SOON-NP(OC) (Repair & Restorati...
1,2,Hamirpur,"2,00,00,000/-","1,00,00,000/-","1,00,00,000/-","4,00,00,000/-",14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40000000.0,2245-02-106-01-SOON-NP(OC) (Repair & Restorati...
2,3,Kangra,-,"3,00,00,000/-",-,"3,00,00,000/-",14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30000000.0,2245-02-106-01-SOON-NP(OC) (Repair & Restorati...
3,4,Kullu,-,"3,00,00,000/-","2,02,25,400/-","5,02,25,400/-",14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50225400.0,2245-02-106-01-SOON-NP(OC) (Repair & Restorati...
4,5,Mandi,-,"3,00,00,000/-",-,"3,00,00,000/-",14-11-2023,Bank Sanction Order 2023-2024.xlsx,14-11-2023,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30000000.0,2245-02-106-01-SOON-NP(OC) (Repair & Restorati...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,8,NaN,NaN,NaN,NaN,"25,00,000/-",01-03-2024,sdrf-sanction-order_2023-24.xlsx,01-03-2024,Mandi,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2500000.0,2245-02-106-01-SOON-NP(OC) (Repair & Restorati...
554,9,NaN,NaN,NaN,NaN,"75,00,000/-",01-03-2024,sdrf-sanction-order_2023-24.xlsx,01-03-2024,Shimla,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7500000.0,2245-02-106-01-SOON-NP(OC) (Repair & Restorati...
556,11,NaN,NaN,NaN,NaN,"35,00,000/-",01-03-2024,sdrf-sanction-order_2023-24.xlsx,01-03-2024,Solan,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3500000.0,2245-02-106-01-SOON-NP(OC) (Repair & Restorati...
557,12,NaN,NaN,NaN,NaN,"39,91,000/-",01-03-2024,sdrf-sanction-order_2023-24.xlsx,01-03-2024,Una,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3991000.0,2245-02-106-01-SOON-NP(OC) (Repair & Restorati...


In [10]:
import pandas as pd
import glob
import os
from pathlib import Path

district_columns = [
    'District',
    'Name of Districts',
    'Name of District',
    'Name of District/Department',
    'Name of Department / Distt.',
    'Distt.',
    'Name of Distt.'
]

def load_and_tag(path):
    """
    Read every sheet in the given Excel file, tag each with its sheet name as 'timeperiod',
    and return the list of DataFrames.
    """
    xls = pd.ExcelFile(path)
    dfs = []
    for sheet in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet)
        df['timeperiod'] = sheet
        dfs.append(df)
    return dfs

def load_all_excels_and_report(folder_path):
    """
    Load every .xls/.xlsx in folder_path, collect all sheets,
    and then print:
      - how many sheets per file have district columns
      - overall percentage of rows containing district info
    """
    excel_paths = glob.glob(os.path.join(folder_path, '*.xls*'))
    all_dfs = []
    grand_with_sheets = 0
    grand_total_sheets = 0

    for path in sorted(excel_paths):
        # load sheets
        dfs = load_and_tag(path)

        # per-file sheet counts
        sheets_with = 0
        for df in dfs:
            if set(df.columns).intersection(district_columns):
                sheets_with += 1
        total_sheets = len(dfs)
        grand_with_sheets += sheets_with
        grand_total_sheets += total_sheets
        print(f"{Path(path).name}: {sheets_with}/{total_sheets} sheets have district columns")

        all_dfs.extend(dfs)

    # concatenate everything
    big_df = pd.concat(all_dfs, ignore_index=True)

    # identify which of your district_columns actually made it in
    found_cols = list(set(big_df.columns).intersection(district_columns))
    if not found_cols:
        print("No district columns found at all!")
        return all_dfs

    # compute row‐level mask
    mask = big_df[found_cols].notna().any(axis=1)
    total_rows = len(big_df)
    rows_with_info = mask.sum()
    pct_with_info = rows_with_info / total_rows * 100

    print()
    print(f"Overall: {grand_with_sheets}/{grand_total_sheets} sheets contain district columns")
    print(f"Across all {total_rows:,} rows, {rows_with_info:,} ({pct_with_info:.2f}%) have district info")

    return all_dfs

# --- USAGE ---
folder = r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables'
all_dataframes = load_all_excels_and_report(folder)


Bank Sanction Order 2023-2024.xlsx: 3/5 sheets have district columns
CB Sanction Order 2023-2024.xlsx: 2/5 sheets have district columns
SDRF-NDRF_2023-24.xlsx: 3/5 sheets have district columns
ndmf-sanction-order_2023-24.xlsx: 3/6 sheets have district columns
ndrf-sanction-order_2022-23.xlsx: 3/6 sheets have district columns
ndrf-sanction-order_2023-24.xlsx: 8/15 sheets have district columns
sdmf-sanction-order_2021-22.xlsx: 0/3 sheets have district columns
sdmf-sanction-order_2022-23.xlsx: 2/3 sheets have district columns
sdrf-sanction-order_2019-2020.xlsx: 4/7 sheets have district columns
sdrf-sanction-order_2020-21.xlsx: 9/16 sheets have district columns
sdrf-sanction-order_2021-22.xlsx: 3/8 sheets have district columns
sdrf-sanction-order_2022-23.xlsx: 5/10 sheets have district columns
sdrf-sanction-order_2023-24.xlsx: 9/16 sheets have district columns

Overall: 54/105 sheets contain district columns
Across all 781 rows, 518 (66.33%) have district info


In [11]:
import os
import glob
import pandas as pd

def load_and_tag(path):
    """Read every sheet in the given Excel file, tag with its sheet name as timeperiod."""
    print(path)
    xls = pd.ExcelFile(path)
    dfs = []
    for sheet in xls.sheet_names:
        print(sheet)
        df = pd.read_excel(xls, sheet_name=sheet)
        df['timeperiod'] = sheet
        dfs.append(df)
    return dfs

def normalize_columns(df):
    """
    # Uncomment and customize this mapping if your sheets use different headers for the same field.
    col_map = {
        'Name of Distt.': 'district',
        'Name of District': 'district',
        'Sr. No.': 'sr_no',
        'Amount (In Crore)': 'amount_cr',
        # ... etc
    }
    return df.rename(columns=col_map)
    """
    return df  # no-op if you don't need normalization

def safe_outer_merge(dfs, on='timeperiod'):
    """If multiple DataFrames share the same timeperiod, outer-merge them,
       but error out if any overlapping non-key column has conflicting values."""
    from functools import reduce

    def merge_two(a, b):
        common = set(a.columns).intersection(b.columns) - {on}
        # find conflicts: for each row-key, column, both non-null and not equal
        merged = pd.merge(a, b, on=on, how='outer', suffixes=('_x','_y'))
        for col in common:
            x, y = col + '_x', col + '_y'
            # where both non-null and unequal
            conflict = merged[x].notna() & merged[y].notna() & (merged[x] != merged[y])
            if conflict.any():
                rows = merged[conflict][[on, x, y]]
                raise ValueError(f"Conflict in timeperiod={merged[on].iloc[0]}, column={col}:\n{rows}")
            # coalesce into single column
            merged[col] = merged[x].combine_first(merged[y])
            merged.drop([x, y], axis=1, inplace=True)
        return merged

    # group by timeperiod
    out = []
    for tp, group in pd.concat(dfs).groupby('timeperiod'):
        if tp == '':
            raise ValueError("Empty timeperiod found in input data")
                
        # if only one DataFrame for this timeperiod, just return it
        if len(group) == 1:
            out.append(group)
        else:
            # outer-merge all frames in this group
            frames = [g.drop(columns='timeperiod') for _, g in group.groupby(group.index)]
            merged = reduce(merge_two, [group[['timeperiod']].iloc[[0]].assign(**{})] + frames)
            out.append(merged)
    return pd.concat(out, ignore_index=True)

def build_master_table(folder):
    all_dfs = []
    for fname in glob.glob(os.path.join(folder, '*.xlsx')):
        all_dfs.extend(load_and_tag(fname))
    # normalize columns if needed
    all_dfs = [normalize_columns(df) for df in all_dfs]
    # try safe outer-merge for repeated timeperiods
    #master = safe_outer_merge(all_dfs)
    return pd.concat(all_dfs, ignore_index=True)

if __name__ == '__main__':
    folder = r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables'
    master_df = build_master_table(folder)
    master_df
    #master_df.to_excel('master_table.xlsx', index=False)
    #print("Master table written to master_table.xlsx")
    


D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables\Bank Sanction Order 2023-2024.xlsx
01-08-2023
23-08-2023
14-11-2023
23-01-2024
04-03-2024
D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables\CB Sanction Order 2023-2024.xlsx
16-09-2023
17-01-2024
31-01-2024
18-03-2024
30-03-2024
D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables\ndmf-sanction-order_2023-24.xlsx
10-04-2023
13-06-2023
11-07-2023
13-12-2023
09-01-2024
30-01-2024
D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables\ndrf-sanction-order_2022-23.xlsx
22-09-2022
22-09-2022_2
22-03-2023
22-03-2023_2
25-03-2023
29-03-2023
D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables\ndrf-sanct

In [13]:
import pandas as pd
import glob
import os

# Folder with your Excel files


excel_folder = r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables'
excel_files = glob.glob(os.path.join(excel_folder, '*.xlsx'))
print(f"Found {len(excel_files)} Excel files in {excel_folder}")
# Initialize an empty list to hold DataFrames  
master_df = []
for file in excel_files:
    # Read the Excel file
    xls = pd.ExcelFile(file,engine='openpyxl')
    # Iterate through each sheet
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet_name)
        print(df)
        # Add a new column with the sheet name
        df['timeperiod'] = sheet_name
        # Append the DataFrame to the list
        master_df.append(df)
master_df

Found 13 Excel files in D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\SDRF\extracted_tables
                      Name of Department  \
0  HPPWD through Engineer-in-Chief HPPWD   
1                                  Total   

                                        Account Head            Amount  
0  2245-02-106-01-SOON-NP(OC) (Repair & Restorati...   Rs. 50,00,000/-  
1                                                NaN  ​Rs. 50,00,000/-  
  Name of District/Department  \
0                      Kangra   
1                      Shimla   
2                       Solan   
3                       Total   

                                     Head of Account         Amount  
0  2245-02-113-01-S00N-NP(OC) Repair/Construction...  1,50,00,000/-  
1  2245-02-113-01-S00N-NP(OC) Repair/Construction...  1,95,00,000/-  
2  2245-02-113-01-S00N-NP(OC) Repair/Construction...  1,50,00,000/-  
3                                                NaN  4,

[                      Name of Department  \
 0  HPPWD through Engineer-in-Chief HPPWD   
 1                                  Total   
 
                                         Account Head            Amount  \
 0  2245-02-106-01-SOON-NP(OC) (Repair & Restorati...   Rs. 50,00,000/-   
 1                                                NaN  ​Rs. 50,00,000/-   
 
    timeperiod  
 0  01-08-2023  
 1  01-08-2023  ,
   Name of District/Department  \
 0                      Kangra   
 1                      Shimla   
 2                       Solan   
 3                       Total   
 
                                      Head of Account         Amount  \
 0  2245-02-113-01-S00N-NP(OC) Repair/Construction...  1,50,00,000/-   
 1  2245-02-113-01-S00N-NP(OC) Repair/Construction...  1,95,00,000/-   
 2  2245-02-113-01-S00N-NP(OC) Repair/Construction...  1,50,00,000/-   
 3                                                NaN  4,95,00,000/-   
 
    timeperiod  
 0  23-08-2023  
 1  23-08-2023  